In [9]:
import aisdb
import nest_asyncio
from aisdb import DBQuery
from aisdb.database.dbconn import PostgresDBConn
from datetime import datetime
from PIL import ImageFile
from aisdb.weather.data_store import WeatherDataStore # for weather

nest_asyncio.apply()
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# >>> PostgreSQL Information <<<
db_user='<username>'            # DB User
db_dbname='aisviz'         # DB Schema
db_password='<password>'    # DB Password
db_hostaddr='127.0.0.1'    # DB Host address

dbconn = PostgresDBConn(
    port=5555,             # PostgreSQL port
    user=db_user,          # PostgreSQL username
    dbname=db_dbname,      # PostgreSQL database
    host=db_hostaddr,      # PostgreSQL address
    password=db_password,  # PostgreSQL password
)

In [11]:
xmin, ymin, xmax, ymax = -70, 45, -58, 53
gulf_bbox = [xmin, xmax, ymin, ymax]
start_time = datetime(2023, 8, 1)
end_time = datetime(2023, 8, 30)

qry = DBQuery(
    dbconn=dbconn,
    start=start_time, end=end_time,
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax,
    callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
)

ais_tracks = []
rowgen = qry.gen_qry()
tracks = aisdb.track_gen.TrackGen(rowgen, decimate=True)

# Initialize WeatherDataStore Instance to work with Weather Grib Files from ERA5

In [ ]:
# If you want to explicitely download weather data from CDS, set download_from_cds=True
weather_data_store = WeatherDataStore(short_names = ['10v','10u'], start = start_time,end =  end_time,weather_data_path = ".",download_from_cds = True,area = [-70, 45, -58, 53])

# If you already have the weather data downloaded, set download_from_cds=False, and specify path to the Grib files.
# weather_data_store = WeatherDataStore(short_names = ['10v','10u'],start = start_time,end = end_time,weather_data_path = "/home/CanadaV2",download_from_cds=True)

2025-05-13 15:17:51,089 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 15:17:51,090 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
[2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
Processing 2023-8: Last day is 31
Final request parameters: {'product_type': ['reanalysis'], 'data_format': 'grib', 'download_format': 'unarchived', 'area': [53, -70, 

2025-05-13 15:17:52,111 INFO Request ID is ce6a71f5-e225-4c68-a604-1aca6ac95a35
Request ID is ce6a71f5-e225-4c68-a604-1aca6ac95a35
2025-05-13 15:17:52,661 INFO status has been updated to accepted
status has been updated to accepted
2025-05-13 15:18:14,542 INFO status has been updated to running
status has been updated to running
2025-05-13 15:19:47,744 INFO status has been updated to successful
status has been updated to successful
91f6fc85dbdf1de5c8ca4185a964fd6b.grib:  24%|██▍       | 136M/564M [29:20<1:27:58, 84.9kB/s] 

In [5]:
tracks = weather_data_store.yield_tracks_with_weather(tracks)

for t in tracks:
    print(f"'u-component' 10m wind for:\nlat: {t['lat'][0]} \nlon: {t['lon'][0]} \ntime: {t['time'][0]} \nis {t['weather_data']['u10'][0]} m/s")
    break

weather_data_store.close()

'u-component' 10m wind for:
lat: 50.003334045410156 
lon: -66.76000213623047 
time: 1690858823 
is 1.5310821533203125 m/s


# Only download Weather Grib Files


In [6]:
from aisdb.weather.weather_fetch import ClimateDataStore # for weather

climateDataStore = ClimateDataStore(dataset="reanalysis-era5-single-levels", short_names= ['10v','10u'], start_time = start_time, end_time = end_time, area= gulf_bbox)
climateDataStore.download_grib_file(output_folder="/weather_data")

2025-05-13 15:14:41,225 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 15:14:41,227 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
[2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
Processing 2023-8: Last day is 31
Final request parameters: {'product_type': ['reanalysis'], 'data_format': 'grib', 'download_format': 'unarchived', 'area': [53, -70, 

Error while fetching weather data: [Errno 13] Permission denied: '/weather_data'
